# Build a Weaviate Personalization Agent - Movie Recommender

In this recipe, we will use the new Weaviate `PersonalizationAgent` to fetch personalized objects from a Weaviate collection, in a user personalized way. This new agentic way of retrieving objects is based on a users persona profile and past interactions with your collection.

> 📚 You can learn more about how to use the `PersonalizationAgent`, in our ["Introducing the Weaviate Personalization Agent"](https://weaviate.io/blog/personalization-agent?utm_source=recipe&utm_campaign=agents) blog and [documentation](https://weaviate.io/developers/agents/personalization?utm_source=recipe&utm_campaign=agents).

To help you get started, we're providing a few demo datasets, available on Hugging Face datasets 🤗:

- [Movies](https://huggingface.co/datasets/weaviate/agents/viewer/personalization-agent-movies): A dataset that lists movies, their ratings, original language etc.
- [Recipes](https://huggingface.co/datasets/weaviate/agents/viewer/personalization-agent-recipes): A dataset that lists the name, short description and cuisine of a dish.

For this example, we will be using the movies dataset to create a movie recommender service

>[Build a Weaviate Personalization Agent - Movie Recommender](#scrollTo=RFe3JtuV6G2f)

>>[Setting up Weaviate & Importing Data](#scrollTo=Kr6AL2OzjqAW)

>>[Setting Up Weaviate & Importing Data](#scrollTo=IIiKq8EmjyTM)

>>>[Create a New Collection](#scrollTo=Vde1N6bb_QH8)

>>[Create a Personalization Agent](#scrollTo=UJ_Fusfj_fw6)

>>>[Adding New Personas](#scrollTo=-_Gcqm9QAF-j)

>>>[Adding Interactions](#scrollTo=3-OxiwY4AU3F)

>>[Get Recommendations and Rationale](#scrollTo=wyiFp4A1Atpr)

>>>[Get Recommendations with an Instruction](#scrollTo=q87tg5hTCVFb)



In [ ]:
!pip install weaviate-client[agents] datasets

## Setting Up Weaviate & Importing Data


To use the Weaviate Personalization Agent, first, create a [Weaviate Cloud](tps://weaviate.io/deployment/serverless?utm_source=recipe&utm_campaign=agents) account👇
1. [Create Serverless Weaviate Cloud account](https://weaviate.io/deployment/serverless?utm_source=recipe&utm_campaign=agents) and setup a free [Sandbox](https://weaviate.io/developers/wcs/manage-clusters/create#sandbox-clusters?utm_source=recipe&utm_campaign=agents)
2. Go to 'Embedding' and enable it, by default, this will make it so that we use `Snowflake/snowflake-arctic-embed-l-v2.0` as the embedding model
3. Take note of the `WEAVIATE_URL` and `WEAVIATE_API_KEY` to connect to your cluster below

> Info: We recommend using [Weaviate Embeddings](https://weaviate.io/developers/weaviate/model-providers/weaviate?utm_source=recipe&utm_campaign=agents) so you do not have to provide any extra keys for external embedding providers.

In [4]:
import os

import weaviate
from weaviate.auth import Auth
from getpass import getpass

if "WEAVIATE_API_KEY" not in os.environ:
  os.environ["WEAVIATE_API_KEY"] = getpass("Weaviate API Key")
if "WEAVIATE_URL" not in os.environ:
  os.environ["WEAVIATE_URL"] = getpass("Weaviate URL")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ.get("WEAVIATE_URL"),
    auth_credentials=Auth.api_key(os.environ.get("WEAVIATE_API_KEY")),
)

### Create a New Collection

Next, we create a new collection in Weaviate called "Movies". For the agentic services in Weaviate, it's a good idea to include descriptions of the properties in your collection. These descriptions can then be used by the agent.

In [5]:
from weaviate.classes.config import Configure, DataType, Property

# if client.collections.exists("Movies"):
    # client.collections.delete("Movies")

client.collections.create(
    "Movies",
    description="A dataset that lists movies, including their release dates, ratings, popularity etc.",
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        Property(
            name="release_date",
            data_type=DataType.TEXT,
            description="release date of the movie",
            skip_vectorization=True,
        ),
        Property(
            name="title", data_type=DataType.TEXT, description="title of the movie"
        ),
        Property(
            name="overview",
            data_type=DataType.TEXT,
            description="overview of the movie",
        ),
        Property(
            name="genres",
            data_type=DataType.TEXT_ARRAY,
            description="genres of the movie",
        ),
        Property(
            name="vote_average",
            data_type=DataType.NUMBER,
            description="vote average of the movie",
        ),
        Property(
            name="vote_count",
            data_type=DataType.INT,
            description="vote count of the movie",
        ),
        Property(
            name="popularity",
            data_type=DataType.NUMBER,
            description="popularity of the movie",
        ),
        Property(
            name="poster_url",
            data_type=DataType.TEXT,
            description="poster path of the movie",
            skip_vectorization=True,
        ),
        Property(
            name="original_language",
            data_type=DataType.TEXT,
            description="Code of the language of the movie",
            skip_vectorization=True,
        ),
    ],
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("weaviate/agents", "personalization-agent-movies", split="train", streaming=True)

movies_collection = client.collections.get("Movies")

with movies_collection.batch.dynamic() as batch:
    for item in dataset:
        batch.add_object(properties=item["properties"])

## Create a Personalization Agent

Below, we create a `PersonalizationAgent` for the `"Movies"` collection. If an agent for this collection already exists, we can simply connect to it.

When creating a new `PeresonalizationAgent`, we can also optioanlly define `user_properties`.

User properties can be anything that may be useful iformation about users that will be added to the agent. In this case, since we are creating a Movie recommender service, we may ask each persona to be added with ther `age`, `favorite_genres` and `languages`.

In [5]:
from weaviate.agents.personalization import PersonalizationAgent

if PersonalizationAgent.exists(client, "Movies"):
  agent = PersonalizationAgent.connect(
          client=client,
          reference_collection="Movies",
      )
else:
  agent = PersonalizationAgent.create(
          client=client,
          reference_collection="Movies",
          user_properties={
            "age": DataType.NUMBER,
            "favorite_genres": DataType.TEXT_ARRAY,
            "languages": DataType.TEXT_ARRAY,
        },
      )



### Adding New Personas

We can add new users with `add_persona`, listing the requested user properties when adding them. Try changing the code block below to represent yourself if you like 👇

In [6]:
from uuid import uuid4
from weaviate.agents.classes import Persona, PersonaInteraction

persona_id = uuid4()
agent.add_persona(
    Persona(
        persona_id=persona_id,
        properties={
              "age": 29,
              "favorite_genres": ["RomCom", "Adventure", "Sci-Fi", "Fantasy"],
              "languages": ["English", "French"],
          },
    )
)

### Adding Interactions

Once we have at least one persona for our agent, we can start adding interactions for that persona. For example, in this movie recommender service, it makes sense to add a personas movie reviews.

Each interaction can have a weight between -1.0 (negative) and 1.0 positive. So, we can add some reviews for a number or films below.

It's a good idea to think about what kind of end application may be forwarding these interactions and have a rule around what each weight might represent. For example:
- 1.0: favorite movie  
- 0.8: user liked the movie
- 0.5: user viewed and did not review the movie
- -0.5: user disliked the movie
- -1.0: user absolutely hated the movie 👎

In [10]:
from uuid import UUID
from weaviate.collections.classes.filters import Filter

reviewed_movies = [
    "Fantastic Beasts and Where to Find Them",
    "The Emoji Movie",
    "Titanic",
    "The Shining",
    "Jumanji",
    "West Side Story",
    "The Shape of Water",
    "Morbius"
]

reviews_dict = {
    movie.properties["title"]: movie
    for movie in movies_collection.query.fetch_objects(
        filters=Filter.by_property("title").contains_any(reviewed_movies), limit=20
    ).objects
}

interactions = [
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["Fantastic Beasts and Where to Find Them"].uuid, weight=0.8
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["The Emoji Movie"].uuid, weight=-0.5
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["Titanic"].uuid, weight=0.5
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["The Shining"].uuid, weight=0.5
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["Jumanji"].uuid, weight=0.8
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["West Side Story"].uuid, weight=-0.5
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["The Shape of Water"].uuid, weight=-1.0
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=reviews_dict["Morbius"].uuid, weight=-1.0
    ),
]

In [11]:
agent.add_interactions(interactions=interactions)

## Get Recommendations and Rationale

Now that we have a persona and some interactions for that persona, we can start getting recommended objects from the agent with `get_objects`. We have two options here: we can set `use_agent_ranking` or not.

When we do not use agent ranking, the returned objects are ranked by classic ML clustering, whereas when we do use it, it will go through an additional re-ranking with an LLM and an optioanl `instruction`.


When we use agent ranking, we can also see the rationale behind the ranking in `ranking_rationale` as we've done below 👇

In [12]:
response = agent.get_objects(persona_id, limit=25, use_agent_ranking=True)

print(response.ranking_rationale)
for i, obj in enumerate(response.objects):
    print(f"*****{i}*****")
    print(obj.properties["title"])
    print(obj.properties["overview"])
    print(obj.properties["genres"])
    print(f"vote_average: {obj.properties['vote_average']}")
    print(obj.properties['poster_url'])

We've placed a spotlight on fantasy and adventure titles given your love for these genres. Movies like 'The Chronicles of Narnia' and 'Jumanji' have been prioritized as they align with your past favorites and preferences. Holiday-themed family films were also considered due to their family-friendly and adventurous nature.
*****0*****
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
Siblings Lucy, Edmund, Susan and Peter step through a magical wardrobe and find the land of Narnia. There, they discover a charming, once peaceful kingdom that has been plunged into eternal winter by the evil White Witch, Jadis. Aided by the wise and magnificent lion, Aslan, the children lead Narnia into a spectacular, climactic battle to be free of the Witch's glacial powers forever.
['Adventure', 'Family', 'Fantasy']
vote_average: 7.1
https://image.tmdb.org/t/p/original/kzJip9vndXYSKQHCgekrgqbnUrA.jpg
*****1*****
The Ewok Adventure
The Towani family civilian shuttlecraft crashes on the forest

### Get Recommendations with an Instruction

Optionally, you can also provide the agent with an instruction too. This allows the agent LLM to have more context as to what kind of recommendations it could make.

It may also make sense to set a higher limit for the initial ranking, and then filter down to a smaller group after the agent ranking as we've done below 👇

In [13]:
response = agent.get_objects(persona_id,
                             limit=100,
                             use_agent_ranking=True,
                             instruction="""Your task is to recommend a diverse set of movies that the user may
                             like based on their fave genres and past interactions. Try to avoid recommending multiple films from within
                             the same cinematic universe.""",
)

print(response.ranking_rationale)
for i, obj in enumerate(response.objects[:20]):
    print(f"*****{i}*****")
    print(obj.properties["title"])
    print(obj.properties["overview"])
    print(obj.properties["genres"])
    print(f"vote_average: {obj.properties['vote_average']}")
    print(obj.properties['poster_url'])

We've highlighted a mix of movies from the user's favorite genres — RomCom, Adventure, Sci-Fi, and Fantasy — ensuring a diverse selection. We've also included some lesser-known gems to provide variety while avoiding multiple entries from the same cinematic universe.
*****0*****
Jumanji: Welcome to the Jungle
The tables are turned as four teenagers are sucked into Jumanji's world - pitted against rhinos, black mambas and an endless variety of jungle traps and puzzles. To survive, they'll play as characters from the game.
['Adventure', 'Action', 'Comedy', 'Fantasy']
vote_average: 6.8
https://image.tmdb.org/t/p/original/pSgXKPU5h6U89ipF7HBYajvYt7j.jpg
*****1*****
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe
Siblings Lucy, Edmund, Susan and Peter step through a magical wardrobe and find the land of Narnia. There, they discover a charming, once peaceful kingdom that has been plunged into eternal winter by the evil White Witch, Jadis. Aided by the wise and magnificent lion,